# Life Style Data


link  https://www.kaggle.com/datasets/jockeroika/life-style-data


Data collection to do recommendations systems and App

## Import's

In [25]:
import kagglehub
import numpy as np
import pandas as pd
import joblib, os
from typing import Dict, Any
import os

## Load The Dataset

In [3]:
# Download latest version
path = kagglehub.dataset_download("jockeroika/life-style-data")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'life-style-data' dataset.
Path to dataset files: /kaggle/input/life-style-data


In [4]:
print(os.listdir(path))

['Final_data.csv', 'meal_metadata.csv']


In [5]:
df = pd.read_csv(os.path.join(path,'Final_data.csv'))

## Explore The Data


In [6]:
df.head()

,Age,Gender,Weight (kg),Height (m),Max_BPM,Avg_BPM,Resting_BPM,Session_Duration (hours),Calories_Burned,Workout_Type,...,cal_from_macros,pct_carbs,protein_per_kg,pct_HRR,pct_maxHR,cal_balance,lean_mass_kg,expected_burn,Burns Calories (per 30 min)_bc,Burns_Calories_Bin
0,34.91,Male,65.27,1.62,188.58,157.65,69.05,1.00,1080.90,Strength,...,2139.59,0.500432,1.624789,0.741237,0.835985,725.10,47.777394,685.1600,7.260425e+19,Medium
1,23.37,Female,56.41,1.55,179.43,131.75,73.18,1.37,1809.91,HIIT,...,1711.65,0.500850,1.514093,0.551247,0.734270,-232.91,40.809803,978.6184,1.020506e+20,High
2,33.20,Female,58.98,1.67,175.04,123.95,54.96,0.91,802.26,Cardio,...,1965.92,0.500610,1.663445,0.574534,0.708124,805.74,44.635580,654.5266,1.079607e+20,High
3,38.69,Female,93.78,1.70,191.21,155.10,50.07,1.10,1450.79,HIIT,...,1627.28,0.499533,0.862017,0.744155,0.811150,1206.21,63.007432,773.6300,8.987921e+19,High
4,45.09,Male,52.42,1.88,193.58,152.88,70.84,1.08,1166.40,Strength,...,2659.23,0.500581,2.538153,0.668405,0.789751,303.60,43.347504,711.4176,5.264685e+19,Low


In [7]:
df.shape

(20000, 54)

In [8]:
df.isnull().sum().sum()

np.int64(0)

In [9]:
df.duplicated().sum()

np.int64(0)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 54 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Age                             20000 non-null  float64
 1   Gender                          20000 non-null  object 
 2   Weight (kg)                     20000 non-null  float64
 3   Height (m)                      20000 non-null  float64
 4   Max_BPM                         20000 non-null  float64
 5   Avg_BPM                         20000 non-null  float64
 6   Resting_BPM                     20000 non-null  float64
 7   Session_Duration (hours)        20000 non-null  float64
 8   Calories_Burned                 20000 non-null  float64
 9   Workout_Type                    20000 non-null  object 
 10  Fat_Percentage                  20000 non-null  float64
 11  Water_Intake (liters)           20000 non-null  float64
 12  Workout_Frequency (days/week)   

## Preprocessing

In [11]:
df['Gender'] = df['Gender'].apply(lambda x:1 if x =="Male" else 0)

In [12]:
df['diet_type'].unique()

array(['Vegan', 'Vegetarian', 'Paleo', 'Keto', 'Low-Carb', 'Balanced'],
      dtype=object)

In [13]:
df['cooking_method'].unique()

array(['Grilled', 'Fried', 'Boiled', 'Baked', 'Steamed', 'Raw', 'Roasted'],
      dtype=object)

In [14]:
drops = ["meal_name",	"meal_type"]

## Config

In [26]:
OUTPUT_DIR = "/kaggle/working" if os.path.exists("/kaggle/working") else "."

# Global Config
ENERGY_CFG = {
    "deficit_low": 0.15,     # overweight cut
    "deficit_high": 0.20,    # obese cut
    "gentle_cut": 0.10,      # healthy recomposition
    "min_deficit": 300,      # clamp absolute kcal deficit
    "max_deficit": 900
}

MACRO_CFG = {
    "protein_g_per_kg": 1.8,     # 1.6–2.2 g/kg realistic for fat-loss preserving muscle
    "carbs_low_g_per_kg": 2.5,   # rest day baseline
    "carbs_high_g_per_kg": 4.0,  # training day bump
    "fat_min_pct": 0.20,         # 20–35% calories from fats
    "fat_max_pct": 0.35,
    "default_meals": 3
}

MEAL_RANK_CFG = {
    "sugar_weight": 0.20,    # penalize sugar relative to calories
    "sodium_weight": 0.10    # penalize sodium vs 2000 mg ref
}

WORKOUT_CFG = {
    "default_days_week": 4,
    "equipment_pref": "Body",       # prefer bodyweight/minimal; change to "Dumbbell"/"Cable" if gym-based
    "difficulty_order": {"Beginner":0, "Intermediate":1, "Advanced":2}
}


## Load Data + Normalize Columns

In [20]:
if "df" not in globals():
    raise RuntimeError("Please create a pandas DataFrame named `df` (e.g., df = pd.read_csv('/content/your.csv'))")

df = df.copy()
df.columns = [c.strip().replace(" ", "_").replace("(", "").replace(")", "").replace("/", "_").replace("-", "_")
              for c in df.columns]

# Quick peek
print("Columns:", len(df.columns))
print(df.head(2).T.head(25))

Columns: 70
                                     0           1
Age                              34.91       23.37
Gender                               1           0
Weight_kg                        65.27       56.41
Height_m                          1.62        1.55
Max_BPM                         188.58      179.43
Avg_BPM                         157.65      131.75
Resting_BPM                      69.05       73.18
Session_Duration_hours             1.0        1.37
Calories_Burned                 1080.9     1809.91
Workout_Type                  Strength        HIIT
Fat_Percentage               26.800377   27.655021
Water_Intake_liters                1.5         1.9
Workout_Frequency_days_week       3.99         4.0
Experience_Level                  2.01        2.01
BMI                              24.87       23.48
Daily_meals_frequency             2.99        3.01
Physical_exercise                 0.01        0.97
Carbs                           267.68      214.32
Proteins           

## Core Helpers (BMI, status, BMR/TDEE, macros)

In [21]:
def compute_bmi(weight_kg, height_m):
    if pd.isna(weight_kg) or pd.isna(height_m) or height_m <= 0: return np.nan
    return weight_kg / (height_m**2)

def status_from_bmi_bodyfat(bmi, fat_pct, gender):
    # BMI band
    if pd.isna(bmi):       bmi_status = "unknown"
    elif bmi < 18.5:       bmi_status = "underweight"
    elif bmi < 25:         bmi_status = "healthy"
    elif bmi < 30:         bmi_status = "overweight"
    else:                  bmi_status = "obese"
    # Sex-aware bodyfat (assumes Gender: 1=male, 0=female)
    if pd.isna(fat_pct) or pd.isna(gender): return bmi_status
    is_male = int(gender) == 1
    if is_male:
        if fat_pct > 25: return "obese"
        if fat_pct >= 20: return "overweight" if bmi_status in ["healthy","overweight"] else bmi_status
    else:
        if fat_pct > 32: return "obese"
        if fat_pct >= 30: return "overweight" if bmi_status in ["healthy","overweight"] else bmi_status
    return bmi_status

def mifflin_st_jeor_bmr(weight_kg, height_m, age, gender):
    if any(pd.isna(x) for x in [weight_kg, height_m, age, gender]) or height_m <= 0: return np.nan
    height_cm = height_m * 100
    is_male = int(gender) == 1
    return 10*weight_kg + 6.25*height_cm - 5*age + (5 if is_male else -161)

def activity_factor(freq_days_week, session_hours):
    # simple, robust mapping from weekly hours
    freq = freq_days_week if pd.notna(freq_days_week) else 4
    dur  = session_hours if pd.notna(session_hours) else 1.0
    weekly_hours = freq * dur
    if weekly_hours < 2:   return 1.35
    if weekly_hours < 4:   return 1.45
    if weekly_hours < 6:   return 1.55
    if weekly_hours < 8:   return 1.65
    return 1.75

def choose_deficit_pct(status: str):
    s = str(status).lower()
    if s == "obese": return ENERGY_CFG["deficit_high"]
    if s == "overweight": return ENERGY_CFG["deficit_low"]
    return ENERGY_CFG["gentle_cut"]

def clamp_calorie_target_from_deficit(tdee, deficit_pct):
    if pd.isna(tdee): return np.nan
    raw = tdee * (1 - float(deficit_pct))
    lo = tdee - ENERGY_CFG["max_deficit"]
    hi = tdee - ENERGY_CFG["min_deficit"]
    if lo > hi: lo, hi = hi, lo
    return float(np.clip(raw, lo, hi))

def macro_targets(weight_kg, cal_target, hi_activity=False):
    if pd.isna(weight_kg) or pd.isna(cal_target) or cal_target <= 0:
        return dict(protein_g=np.nan, carbs_g=np.nan, fat_g=np.nan, total_kcal=np.nan)
    p_g = MACRO_CFG["protein_g_per_kg"] * weight_kg
    c_g = (MACRO_CFG["carbs_high_g_per_kg"] if hi_activity else MACRO_CFG["carbs_low_g_per_kg"]) * weight_kg
    remaining = cal_target - (4*p_g + 4*c_g)
    fat_pct = np.clip(remaining / cal_target if cal_target > 0 else MACRO_CFG["fat_min_pct"],
                      MACRO_CFG["fat_min_pct"], MACRO_CFG["fat_max_pct"])
    f_kcal = fat_pct * cal_target
    f_g = f_kcal / 9.0
    total_kcal = 4*p_g + 4*c_g + 9*f_g
    return dict(protein_g=p_g, carbs_g=c_g, fat_g=f_g, total_kcal=total_kcal)

def per_meal_split(total_macros, meals_per_day):
    meals = int(meals_per_day) if pd.notna(meals_per_day) and meals_per_day > 0 else MACRO_CFG["default_meals"]
    return {
        "meals_per_day": meals,
        "permeal_protein_g": total_macros["protein_g"]/meals,
        "permeal_carbs_g":   total_macros["carbs_g"]/meals,
        "permeal_fat_g":     total_macros["fat_g"]/meals,
        "permeal_calories":  total_macros["total_kcal"]/meals
    }


## Meal & Workout Utilities

In [22]:
def meal_score(row, target):
    # target: dict with protein, carbs, fat, cal (per meal)
    d_p = (row.get("Proteins",0) - target["protein"])**2
    d_c = (row.get("Carbs",0)   - target["carbs"])**2
    d_f = (row.get("Fats",0)    - target["fat"])**2
    d_cal = (row.get("Calories",0) - target["cal"])**2
    macro_dist = np.sqrt(d_p + d_c + d_f + 0.25*d_cal)
    sugar_pen  = MEAL_RANK_CFG["sugar_weight"]  * (row.get("sugar_g",0) / max(target["cal"],1))
    sodium_pen = MEAL_RANK_CFG["sodium_weight"] * (row.get("sodium_mg",0) / 2000.0)
    return macro_dist + sugar_pen + sodium_pen

def best_meal(df_all, meal_type, target, diet_pref=None):
    pool = df_all.copy()
    if "meal_type" in pool.columns:
        pool = pool[pool["meal_type"].astype(str).str.lower()==meal_type.lower()]
    if diet_pref is not None and "diet_type" in pool.columns:
        pool = pool[pool["diet_type"].astype(str).str.lower()==str(diet_pref).lower()]
    need = {"meal_name","diet_type","Carbs","Proteins","Fats","Calories","sugar_g","sodium_mg"}
    if len(pool)==0 or not need.issubset(set(pool.columns)):
        return {}
    t = {"protein": target["permeal_protein_g"], "carbs": target["permeal_carbs_g"],
         "fat": target["permeal_fat_g"], "cal": target["permeal_calories"]}
    pool = pool.copy()
    pool["_score"] = pool.apply(lambda r: meal_score(r, t), axis=1)
    row = pool.sort_values("_score").head(1)
    if len(row)==0: return {}
    r = row.iloc[0]
    return {
        "meal_name": r.get("meal_name",""),
        "meal_cal": float(r.get("Calories", np.nan)),
        "meal_P": float(r.get("Proteins", np.nan)),
        "meal_C": float(r.get("Carbs", np.nan)),
        "meal_F": float(r.get("Fats", np.nan)),
        "diet_type": r.get("diet_type",""),
        "meal_type": meal_type
    }

def map_experience_to_diff(exp):
    if pd.isna(exp): return "Beginner"
    if exp <= 1.2:   return "Beginner"
    if exp <= 2.0:   return "Intermediate"
    return "Advanced"

def select_workouts_pool(df_all, target_diff, equipment_pref=None):
    q = df_all.copy()
    if "Difficulty_Level" in q.columns:
        order = WORKOUT_CFG["difficulty_order"]; tgt = order.get(target_diff, 0)
        q = q[q["Difficulty_Level"].map(order).fillna(2) <= tgt]
    if equipment_pref and "Equipment_Needed" in q.columns:
        mask = q["Equipment_Needed"].astype(str).str.contains(str(equipment_pref), case=False)
        mask |= q["Equipment_Needed"].astype(str).str.contains("Body|None|Mat", case=False)
        q = q[mask]
    if "Burns_Calories_per_30_min" in q.columns:
        q = q.sort_values("Burns_Calories_per_30_min", ascending=False)
    return q.reset_index(drop=True)

def hrmax_est(age, max_bpm_col):
    if pd.notna(max_bpm_col) and max_bpm_col > 100: return float(max_bpm_col)
    if pd.notna(age): return float(220 - age)
    return 190.0

## Planner: Diet-Only vs Diet+Workout (different calories/macros)

In [23]:
def core_status_energy(r: pd.Series):
    bmi = r.get("BMI_calc", np.nan)
    if pd.isna(bmi):
        bmi = compute_bmi(r.get("Weight_kg", np.nan), r.get("Height_m", np.nan))
    status = status_from_bmi_bodyfat(bmi, r.get("Fat_Percentage", np.nan), r.get("Gender", np.nan))
    bmr = mifflin_st_jeor_bmr(r.get("Weight_kg", np.nan), r.get("Height_m", np.nan), r.get("Age", np.nan), r.get("Gender", np.nan))
    return bmi, status, bmr

def tdee_from_activity(r: pd.Series, override_activity_factor=None):
    if override_activity_factor is not None:
        return r.get("BMR_cache", np.nan) * override_activity_factor
    af  = activity_factor(r.get("Workout_Frequency_days_week", np.nan), r.get("Session_Duration_hours", np.nan))
    return r.get("BMR_cache", np.nan) * af

def expected_session_burn(row_ex, session_hours):
    per30 = float(row_ex.get("Burns_Calories_per_30_min", 0.0))
    dur = session_hours if pd.notna(session_hours) and session_hours > 0 else 1.0
    return per30 * (dur / 0.5)

def build_workout_choice(r: pd.Series, df_all: pd.DataFrame):
    target_diff = map_experience_to_diff(r.get("Experience_Level", np.nan))
    pool = select_workouts_pool(df_all, target_diff, WORKOUT_CFG["equipment_pref"])
    if len(pool)==0: return {}, 0.0
    w = pool.iloc[0]  # top burn within constraints
    burn = expected_session_burn(w, r.get("Session_Duration_hours", np.nan))
    return w, burn

def plan_diet_only(r: pd.Series):
    # No extra workout calories; AF from existing frequency/duration still applies (light activity).
    deficit = choose_deficit_pct(r.get("weight_status_cache", "healthy"))
    tdee = r.get("TDEE_base_cache", np.nan)
    cal_target = clamp_calorie_target_from_deficit(tdee, deficit)
    hi_act = str(r.get("Workout_Type","")).lower() in ["hiit","cardio"]
    totals = macro_targets(r.get("Weight_kg", np.nan), cal_target, hi_activity=hi_act)
    permeal = per_meal_split(totals, r.get("Daily_meals_frequency", np.nan))
    return {"mode": "diet_only", "deficit_pct": deficit, "cal_target": cal_target, "totals": totals, "permeal": permeal}

def plan_with_workout(r: pd.Series, df_all: pd.DataFrame):
    # Pick a workout → add its session burn to TDEE for training-day calories (carb bump)
    w, session_burn = build_workout_choice(r, df_all)
    deficit = choose_deficit_pct(r.get("weight_status_cache", "healthy"))
    base_tdee = r.get("TDEE_base_cache", np.nan)
    # Rest-day target (no session burn)
    cal_rest = clamp_calorie_target_from_deficit(base_tdee, deficit)
    # Training-day target (base + session burn)
    cal_train = clamp_calorie_target_from_deficit(base_tdee + session_burn, deficit)

    # Macros: rest → low carbs; train → high carbs
    totals_rest  = macro_targets(r.get("Weight_kg", np.nan), cal_rest,  hi_activity=False)
    totals_train = macro_targets(r.get("Weight_kg", np.nan), cal_train, hi_activity=True)
    permeal_rest  = per_meal_split(totals_rest,  r.get("Daily_meals_frequency", np.nan))
    permeal_train = per_meal_split(totals_train, r.get("Daily_meals_frequency", np.nan))

    return {
        "mode": "with_workout",
        "workout_row": w.to_dict() if isinstance(w, pd.Series) else {},
        "session_burn": float(session_burn),
        "deficit_pct": deficit,
        "cal_rest": cal_rest, "cal_train": cal_train,
        "totals_rest": totals_rest, "totals_train": totals_train,
        "permeal_rest": permeal_rest, "permeal_train": permeal_train
    }

## Recommender Class + Inference (mode="diet_only" or "with_workout")

In [24]:
class DietWorkoutRecommender:
    def __init__(self):
        self.columns_ = None

    def fit(self, df_all: pd.DataFrame):
        self.columns_ = df_all.columns.tolist()
        return self

    def _status_energy_cache(self, r: pd.Series):
        bmi, status, bmr = core_status_energy(r)
        # cache fields onto a copy
        rc = r.copy()
        rc["BMI_calc_cache"] = bmi
        rc["weight_status_cache"] = status
        rc["BMR_cache"] = bmr
        rc["TDEE_base_cache"] = tdee_from_activity(rc)
        return rc

    def _meals_for_targets(self, df_all, permeal, diet_pref):
        target = {
            "permeal_protein_g": permeal["permeal_protein_g"],
            "permeal_carbs_g": permeal["permeal_carbs_g"],
            "permeal_fat_g": permeal["permeal_fat_g"],
            "permeal_calories": permeal["permeal_calories"]
        }
        return {
            "Breakfast": best_meal(df_all, "Breakfast", target, diet_pref),
            "Lunch":     best_meal(df_all, "Lunch",     target, diet_pref),
            "Dinner":    best_meal(df_all, "Dinner",    target, diet_pref),
        }

    def predict_one(self, r: pd.Series, df_all: pd.DataFrame, mode: str = "with_workout") -> Dict[str, Any]:
        rc = self._status_energy_cache(r)
        diet_pref = r.get("diet_type", None)

        if mode == "diet_only":
            plan = plan_diet_only(rc)
            meals = self._meals_for_targets(df_all, plan["permeal"], diet_pref)
            out = {
                # inputs
                "Age": r.get("Age", np.nan), "Gender": r.get("Gender", np.nan),
                "Weight_kg": r.get("Weight_kg", np.nan), "Height_m": r.get("Height_m", np.nan),
                # status/energy
                "BMI_calc": rc["BMI_calc_cache"], "weight_status": rc["weight_status_cache"],
                "BMR": rc["BMR_cache"], "TDEE": rc["TDEE_base_cache"],
                "mode": "diet_only", "deficit_pct": plan["deficit_pct"],
                "calorie_target": plan["cal_target"],
                # daily + per-meal
                "protein_g": plan["totals"]["protein_g"], "carbs_g": plan["totals"]["carbs_g"], "fat_g": plan["totals"]["fat_g"],
                "permeal_calories": plan["permeal"]["permeal_calories"],
                "permeal_protein_g": plan["permeal"]["permeal_protein_g"],
                "permeal_carbs_g": plan["permeal"]["permeal_carbs_g"],
                "permeal_fat_g": plan["permeal"]["permeal_fat_g"],
                # meals
                "Breakfast_name": meals["Breakfast"].get("meal_name",""),
                "Lunch_name": meals["Lunch"].get("meal_name",""),
                "Dinner_name": meals["Dinner"].get("meal_name",""),
            }
            return out

        # with_workout
        plan = plan_with_workout(rc, df_all)
        meals_rest  = self._meals_for_targets(df_all, plan["permeal_rest"],  diet_pref)
        meals_train = self._meals_for_targets(df_all, plan["permeal_train"], diet_pref)

        w = plan["workout_row"]
        return {
            # inputs
            "Age": r.get("Age", np.nan), "Gender": r.get("Gender", np.nan),
            "Weight_kg": r.get("Weight_kg", np.nan), "Height_m": r.get("Height_m", np.nan),
            # status/energy
            "BMI_calc": rc["BMI_calc_cache"], "weight_status": rc["weight_status_cache"],
            "BMR": rc["BMR_cache"], "TDEE": rc["TDEE_base_cache"],
            "mode": "with_workout", "deficit_pct": plan["deficit_pct"],
            # calories/macros (both days)
            "cal_rest": plan["cal_rest"], "cal_train": plan["cal_train"],
            "protein_rest_g": plan["totals_rest"]["protein_g"], "carbs_rest_g": plan["totals_rest"]["carbs_g"], "fat_rest_g": plan["totals_rest"]["fat_g"],
            "protein_train_g": plan["totals_train"]["protein_g"], "carbs_train_g": plan["totals_train"]["carbs_g"], "fat_train_g": plan["totals_train"]["fat_g"],
            # per-meal (both days)
            "permeal_rest_cal": plan["permeal_rest"]["permeal_calories"],
            "permeal_rest_P": plan["permeal_rest"]["permeal_protein_g"],
            "permeal_rest_C": plan["permeal_rest"]["permeal_carbs_g"],
            "permeal_rest_F": plan["permeal_rest"]["permeal_fat_g"],
            "permeal_train_cal": plan["permeal_train"]["permeal_calories"],
            "permeal_train_P": plan["permeal_train"]["permeal_protein_g"],
            "permeal_train_C": plan["permeal_train"]["permeal_carbs_g"],
            "permeal_train_F": plan["permeal_train"]["permeal_fat_g"],
            # meals (one best each)
            "Breakfast_rest":  meals_rest["Breakfast"].get("meal_name",""),
            "Lunch_rest":      meals_rest["Lunch"].get("meal_name",""),
            "Dinner_rest":     meals_rest["Dinner"].get("meal_name",""),
            "Breakfast_train": meals_train["Breakfast"].get("meal_name",""),
            "Lunch_train":     meals_train["Lunch"].get("meal_name",""),
            "Dinner_train":    meals_train["Dinner"].get("meal_name",""),
            # workout chosen + its burn
            "Exercise": w.get("Name_of_Exercise",""),
            "Exercise_type": w.get("Workout_Type",""),
            "Body_part": w.get("Body_Part",""),
            "Difficulty": w.get("Difficulty_Level",""),
            "Equipment": w.get("Equipment_Needed",""),
            "Burn_per_30min": float(w.get("Burns_Calories_per_30_min", np.nan)),
            "Expected_session_burn": plan["session_burn"],
        }

    def predict_batch(self, df_users: pd.DataFrame, df_all: pd.DataFrame, n_rows: int = None, mode: str = "with_workout") -> pd.DataFrame:
        it = df_users.head(n_rows).iterrows() if n_rows else df_users.iterrows()
        rows = [self.predict_one(r, df_all, mode=mode) for _, r in it]
        return pd.DataFrame(rows)

## Fit/Save/Load + Predict with this (writes to /kaggle/working)

In [27]:
reco = DietWorkoutRecommender().fit(df)

# Save model to working dir
MODEL_PATH = os.path.join(OUTPUT_DIR, "diet_workout_recommender.joblib")
joblib.dump({"model": reco, "version": "1.0"}, MODEL_PATH)

# Load to demonstrate (optional)
bundle = joblib.load(MODEL_PATH)
loaded = bundle["model"]

# Choose mode: "diet_only" OR "with_workout"
MODE = "with_workout"      # change to "diet_only" if no training

PRED_ROWS = 10
pred = loaded.predict_batch(df, df, n_rows=PRED_ROWS, mode=MODE)

# Show result in columns (works fine in Kaggle notebooks)
print(f"Mode: {MODE} | Showing first {PRED_ROWS} rows")
try:
    display(pred.head(PRED_ROWS))
except Exception:
    print(pred.head(PRED_ROWS).to_string(index=False))

# Export predictions to working dir (visible in Kaggle "Output" tab)
PRED_PATH = os.path.join(OUTPUT_DIR, f"recommendations_{MODE}.csv")
pred.to_csv(PRED_PATH, index=False)

print("Saved model:", MODEL_PATH)
print("Saved predictions:", PRED_PATH)


Mode: with_workout | Showing first 10 rows


,Age,Gender,Weight_kg,Height_m,BMI_calc,weight_status,BMR,TDEE,mode,deficit_pct,...,Breakfast_train,Lunch_train,Dinner_train,Exercise,Exercise_type,Body_part,Difficulty,Equipment,Burn_per_30min,Expected_session_burn
0,34.91,1,65.27,1.62,24.870447,obese,1495.65,2168.6925,with_workout,0.20,...,Other,Other,Other,Reverse Lunges,Strength,Forearms,Intermediate,None or Dumbbell,381.23,762.4600
1,23.37,0,56.41,1.55,23.479709,healthy,1255.00,1945.2500,with_workout,0.10,...,Other,Other,Other,Reverse Lunges,Strength,Forearms,Intermediate,None or Dumbbell,381.23,1044.5702
2,33.20,0,58.98,1.67,21.148123,healthy,1306.55,1894.4975,with_workout,0.10,...,Other,Other,Other,Dragon Flags,Cardio,Forearms,Beginner,None or Dumbbell,379.47,690.6354
3,38.69,0,93.78,1.70,32.449827,obese,1645.85,2551.0675,with_workout,0.20,...,Other,Other,Other,Reverse Lunges,Strength,Forearms,Intermediate,None or Dumbbell,381.23,838.7060
4,45.09,1,52.42,1.88,14.831372,underweight,1478.75,2292.0625,with_workout,0.10,...,Other,Other,Other,Reverse Lunges,Strength,Forearms,Intermediate,None or Dumbbell,381.23,823.4568
5,53.19,0,105.05,1.84,31.028474,obese,1773.55,2571.6475,with_workout,0.20,...,Other,Other,Other,Dragon Flags,Cardio,Forearms,Beginner,None or Dumbbell,379.47,523.6686
6,23.17,1,58.41,1.78,18.435172,underweight,1585.75,2775.0625,with_workout,0.10,...,Other,Other,Other,Reverse Lunges,Strength,Forearms,Intermediate,None or Dumbbell,381.23,1273.3082
7,55.92,0,84.07,1.63,31.642139,obese,1418.85,2199.2175,with_workout,0.20,...,Other,Other,Other,Reverse Lunges,Strength,Forearms,Intermediate,None or Dumbbell,381.23,770.0846
8,24.11,0,88.58,1.79,27.645829,overweight,1723.00,2842.9500,with_workout,0.15,...,Other,Other,Other,Reverse Lunges,Strength,Forearms,Intermediate,None or Dumbbell,381.23,1341.9296
9,39.19,0,50.32,1.60,19.656250,healthy,1146.25,1662.0625,with_workout,0.10,...,Other,Other,Other,Dragon Flags,Cardio,Forearms,Beginner,None or Dumbbell,379.47,887.9598


Saved model: ./diet_workout_recommender.joblib
Saved predictions: ./recommendations_with_workout.csv


## Diet Recommender

In [34]:
# === PATCH: cleaner display + flexible return ===
import pandas as pd
import numpy as np
import os

OUT_DIR = "/kaggle/working" if os.path.exists("/kaggle/working") else "."

def get_recommendation(
    mode: str = "diet_only",
    age: int = 25,
    gender: int = 1,
    weight: float = 70.0,
    height: float = 1.75,
    fat_pct: float = 25.0,
    experience: float = 1.5,
    workout_freq: int = 3,
    session_hours: float = 1.0,
    workout_type: str = "Cardio",
    daily_meals: int = 3,
    diet_type: str = "Vegan",
    return_mode: str = "df"   # 'df' | 'path' | 'none'
):
    """
    Generate recommendation, save CSV, and display as a table.
    return_mode:
      - 'df'   -> return the DataFrame (so Jupyter shows a table nicely)
      - 'path' -> return only the CSV path (no tuple)
      - 'none' -> return nothing
    """
    try:
        reco
    except NameError:
        raise RuntimeError("Model not found: run the earlier cells to define/fit DietWorkoutRecommender().")

    user_row = pd.Series({
        "Age": age, "Gender": gender, "Weight_kg": weight, "Height_m": height,
        "Fat_Percentage": fat_pct, "Experience_Level": experience,
        "Workout_Frequency_days_week": workout_freq, "Session_Duration_hours": session_hours,
        "Workout_Type": workout_type, "Daily_meals_frequency": daily_meals, "diet_type": diet_type
    })

    out = reco.predict_one(user_row, df, mode=mode)

    def _safe(x, nd=2):
        try: return round(float(x), nd)
        except: return np.nan

    if mode == "diet_only":
        summary = pd.DataFrame([{
            "BMI": _safe(out.get("BMI_calc")), "Weight Status": out.get("weight_status"),
            "BMR": _safe(out.get("BMR")), "TDEE": _safe(out.get("TDEE")),
            "Target Calories": _safe(out.get("calorie_target"))
        }])
        macros = pd.DataFrame([{
            "Protein (g/day)": _safe(out.get("protein_g")),
            "Carbs (g/day)": _safe(out.get("carbs_g")),
            "Fat (g/day)": _safe(out.get("fat_g")),
            "Per Meal Protein": _safe(out.get("permeal_protein_g")),
            "Per Meal Carbs": _safe(out.get("permeal_carbs_g")),
            "Per Meal Fat": _safe(out.get("permeal_fat_g"))
        }])
        meals = pd.DataFrame([{
            "Breakfast": out.get("Breakfast_name", out.get("Breakfast", "(no match)")),
            "Lunch": out.get("Lunch_name", out.get("Lunch", "(no match)")),
            "Dinner": out.get("Dinner_name", out.get("Dinner", "(no match)")),
        }])
        final = pd.concat([summary, macros, meals], axis=1)
        out_path = os.path.join(OUT_DIR, "recommendation_diet_only.csv")

    else:
        summary = pd.DataFrame([{
            "BMI": _safe(out.get("BMI_calc")), "Weight Status": out.get("weight_status"),
            "BMR": _safe(out.get("BMR")), "TDEE": _safe(out.get("TDEE")),
            "Cal Rest": _safe(out.get("cal_rest")), "Cal Train": _safe(out.get("cal_train")),
            "Workout": out.get("Exercise", "(no match)"),
            "Expected Burn": _safe(out.get("Expected_session_burn"))
        }])
        macros = pd.DataFrame([{
            "Protein Rest (g)": _safe(out.get("protein_rest_g")),
            "Carbs Rest (g)": _safe(out.get("carbs_rest_g")),
            "Fat Rest (g)": _safe(out.get("fat_rest_g")),
            "Protein Train (g)": _safe(out.get("protein_train_g")),
            "Carbs Train (g)": _safe(out.get("carbs_train_g")),
            "Fat Train (g)": _safe(out.get("fat_train_g"))
        }])
        meals = pd.DataFrame([{
            "Breakfast (Rest)": out.get("Breakfast_rest", "(no match)"),
            "Lunch (Rest)": out.get("Lunch_rest", "(no match)"),
            "Dinner (Rest)": out.get("Dinner_rest", "(no match)"),
            "Breakfast (Train)": out.get("Breakfast_train", "(no match)"),
            "Lunch (Train)": out.get("Lunch_train", "(no match)"),
            "Dinner (Train)": out.get("Dinner_train", "(no match)"),
        }])
        final = pd.concat([summary, macros, meals], axis=1)
        out_path = os.path.join(OUT_DIR, "recommendation_with_workout.csv")

    # Save CSV
    final.to_csv(out_path, index=False)
    print(f"✅ Saved recommendation to: {out_path}")

    # Display nicely
    try:
        display(final)
    except:
        print(final.to_string(index=False))

    # Return in the chosen format (avoid tuples!)
    if return_mode == "df":
        return final
    elif return_mode == "path":
        return out_path
    else:  # 'none'
        return



## Example

In [35]:
# 1) Show nice table & get a DataFrame back (default)
df_out = get_recommendation(mode="diet_only", age=27, gender=1, weight=72, height=1.76, fat_pct=22, daily_meals=3, diet_type="Vegetarian")

# 2) Only return the path (no tuple in output)
csv_path = get_recommendation(mode="with_workout", age=27, gender=1, weight=72, height=1.76,
                              fat_pct=22, experience=1.8, workout_freq=4, session_hours=1.0,
                              workout_type="Strength", diet_type="Vegetarian", return_mode="path")

# 3) Just display it, return nothing
get_recommendation(mode="diet_only", age=30, gender=0, weight=60, height=1.62, fat_pct=28, return_mode="none")


✅ Saved recommendation to: ./recommendation_diet_only.csv


,BMI,Weight Status,BMR,TDEE,Target Calories,Protein (g/day),Carbs (g/day),Fat (g/day),Per Meal Protein,Per Meal Carbs,Per Meal Fat,Breakfast,Lunch,Dinner
0,23.24,overweight,1690.0,2450.5,2082.92,129.6,288.0,46.29,43.2,96.0,15.43,Other,Other,Other


✅ Saved recommendation to: ./recommendation_with_workout.csv


,BMI,Weight Status,BMR,TDEE,Cal Rest,Cal Train,Workout,Expected Burn,Protein Rest (g),Carbs Rest (g),Fat Rest (g),Protein Train (g),Carbs Train (g),Fat Train (g),Breakfast (Rest),Lunch (Rest),Dinner (Rest),Breakfast (Train),Lunch (Train),Dinner (Train)
0,23.24,overweight,1690.0,2619.5,2226.57,2874.67,Reverse Lunges,762.46,129.6,180.0,86.59,129.6,288.0,111.79,Other,Other,Other,Other,Other,Other


✅ Saved recommendation to: ./recommendation_diet_only.csv


,BMI,Weight Status,BMR,TDEE,Target Calories,Protein (g/day),Carbs (g/day),Fat (g/day),Per Meal Protein,Per Meal Carbs,Per Meal Fat,Breakfast,Lunch,Dinner
0,22.86,healthy,1301.5,1887.17,1587.17,108.0,240.0,35.27,36.0,80.0,11.76,Other,Other,Other
